# Test 3: Preselection
Just test my preselection functions by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

### 2-1. Check the number of events for each branch

In [3]:
if GP.length == Jet.length == Event.length:
    print("There are {} events in the .root file.".format(GP.length))
else:
    print("""\033[7;31m****** There is the problem for the number of event \
    in the .root file. ******\033[0m""")
#     print("\033[7;31m****** There is the problem for the number of event "
#           "in the .root file. ******\033[0m")
    print("\033[7;31m****** Please check your .root file. ******\033[0m")

There are 19373 events in the .root file.


### 2-2. Define physical quantites
Skip this step

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels
Skip this step

## 4. Jet Clustering

### 4-1. Select stable final state particle without/with filtering out dark sector

In [4]:
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [5]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v3 = myAnal_v3.jetClustering_v1(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering_v1(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done new version


## 5. Analyze the Jets in the Truth Level

### 5-1. Preselection version 1

In [6]:
presel_bef, presel_pt, presel_pt_eta, presel_idx = myAnal_v3.preselection_v1(PseudoJet_filterDM_v3, pT_min=20, eta_max=2.5)

19373 events before preselection
19373 events after pT preselection
19373 events after pT & eta preselections
--------------------------------------------------------------------------------
0 events without PseudoJet before preselection
357 events without PseudoJet after pT preselection
519 events without PseudoJet after pT & eta preselections


In [7]:
presel_pt_eta[21]

array([(754.72107953, -0.26936706, -0.35238845, 73.17201984),
       (104.83282762, -1.11482256,  1.14303952, 20.66261802),
       ( 85.25366785,  0.91831199, -2.53096755, 16.94153709),
       ( 54.1991487 ,  0.23044176, -2.59317889, 12.15247457),
       ( 27.54988231, -0.93685402,  0.65517858,  4.1833831 )],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

## 6. Test 1

In [8]:
# print top six PseudoJet in 21th event
PseudoJet_filterDM_v3[21][:7]

[PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172),
 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663),
 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942),
 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325),
 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152),
 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183),
 PseudoJet(pt=19.967, eta=-0.716, phi=-2.708, mass=3.603)]

In [9]:
# print pt > 20 GeV PseudoJet in 21th and 23th events
for i in range(21,23):
    print("event {}".format(i))
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        if jet.pt > 20:
            print(j, jet)
    print('-'*80)

event 21
0 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
1 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
2 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
3 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325)
4 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
5 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)
--------------------------------------------------------------------------------
event 22
0 PseudoJet(pt=135.815, eta=2.146, phi=-3.118, mass=28.584)
1 PseudoJet(pt=23.674, eta=2.146, phi=-2.674, mass=8.452)
2 PseudoJet(pt=23.255, eta=3.290, phi=0.439, mass=4.790)
--------------------------------------------------------------------------------


#### A. After $p_T > 20$ GeV selection

In [10]:
# test after pT selection
_presel_events_pt = []
_num_events_bef, _num_events_after_pt, _num_events_after_eta = [], [], []
_pt, _eta, _phi, m = [], [], [], []

for i in range(21,23):
    print("event {}".format(i))
    _num_events_bef.append(len(PseudoJet_filterDM_v3[i]))
    _event_i = []
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        if jet.pt > 20:
            print(j, jet)
            # print(j, jet, [jet.pt, jet.eta])  # [] = () in np.array()
            # print(j, jet, (jet.pt, jet.eta))
            # sum_test = np.sum(np.array([jet.pt, jet.eta, jet.phi, jet.mass]) -
                            #   np.array((jet.pt, jet.eta, jet.phi, jet.mass)))
            # print('******', sum_test)  # = 0
            _event_i.append((jet.pt, jet.eta, jet.phi, jet.mass))
    print('*'*20 + 'NumPy Array')
    # print('******', _event_i)  # [(pt1, eta1, phi1, m1), (pt2, eta2, ...), ...]
    arr_event_i = np.array(_event_i, dtype=[('pT', '<f8'), ('eta', '<f8'),
                                            ('phi', '<f8'), ('mass', '<f8')])
    print(arr_event_i.shape, arr_event_i)
    _num_events_after_pt.append(len(arr_event_i))
    _presel_events_pt.append(arr_event_i)
    print('-'*80)
    
print("Number of events before preselection = {}".format(_num_events_bef))
print("Number of events after pT preselection = {}".format(_num_events_after_pt))
print("Number of events after eta preselection = {}".format(_num_events_after_eta))
print('='*80)
_presel_events_pt[1]

event 21
0 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
1 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
2 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
3 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325)
4 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
5 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)
********************NumPy Array
(6,) [(754.72107953, -0.26936706, -0.35238845, 73.17201984)
 (104.83282762, -1.11482256,  1.14303952, 20.66261802)
 ( 85.25366785,  0.91831199, -2.53096755, 16.94153709)
 ( 64.97417129, -3.03388869,  1.60866641, 11.32521937)
 ( 54.1991487 ,  0.23044176, -2.59317889, 12.15247457)
 ( 27.54988231, -0.93685402,  0.65517858,  4.1833831 )]
--------------------------------------------------------------------------------
event 22
0 PseudoJet(pt=135.815, eta=2.146, phi=-3.118, mass=28.584)
1 PseudoJet(pt=23.674, eta=2.146, phi=-2.674, mass=8.452)
2 PseudoJet(pt=23.255, eta=3.290, phi=0.439, mass=4.790)
**

array([(135.81475624, 2.14646234, -3.11762065, 28.58375223),
       ( 23.67438139, 2.14635282, -2.67445185,  8.45196396),
       ( 23.25535698, 3.28953198,  0.43855045,  4.79000333)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [26]:
# check myAnal_v3.preselection_v1 result
event_location = [21, 22]

for a, b in enumerate(event_location):
    print("event", a, b)
    sum_pt = np.sum(_presel_events_pt[a]['pT'] - presel_pt[b]['pT'])
    sum_eta = np.sum(_presel_events_pt[a]['eta'] - presel_pt[b]['eta'])
    sum_phi = np.sum(_presel_events_pt[a]['phi'] - presel_pt[b]['phi'])
    sum_m = np.sum(_presel_events_pt[a]['mass'] - presel_pt[b]['mass'])
    print(sum_pt, sum_eta, sum_phi, sum_m)
    print('-'*80)

event 0 21
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------
event 1 22
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------


##### Conclusion: The result of `preselection_v1` is the same as test version of event 21 and 22.

#### B. Before selection

In [28]:
# test before selection
_presel_events_bef = []
_num_events_bef = []

for i in range(21,23):
    print("event {}".format(i))
    _num_events_bef.append(len(PseudoJet_filterDM_v3[i]))
    _event_i = []
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        # print(j, jet)
        _event_i.append((jet.pt, jet.eta, jet.phi, jet.mass))
    print('*'*20 + 'NumPy Array')
    arr_event_i = np.array(_event_i, dtype=[('pT', '<f8'), ('eta', '<f8'),
                                            ('phi', '<f8'), ('mass', '<f8')])
    print(arr_event_i.shape, arr_event_i[:4])
    _presel_events_bef.append(arr_event_i)
    print('-'*80)
    
print("Number of events before preselection = {}".format(_num_events_bef))
print('='*80)
_presel_events_bef[0][:2]

event 21
********************NumPy Array
(75,) [(754.72107953, -0.26936706, -0.35238845, 73.17201984)
 (104.83282762, -1.11482256,  1.14303952, 20.66261802)
 ( 85.25366785,  0.91831199, -2.53096755, 16.94153709)
 ( 64.97417129, -3.03388869,  1.60866641, 11.32521937)]
--------------------------------------------------------------------------------
event 22
********************NumPy Array
(185,) [(135.81475624, 2.14646234, -3.11762065, 28.58375223)
 ( 23.67438139, 2.14635282, -2.67445185,  8.45196396)
 ( 23.25535698, 3.28953198,  0.43855045,  4.79000333)
 ( 19.44977872, 0.22853662,  0.30315499,  4.32690473)]
--------------------------------------------------------------------------------
Number of events before preselection = [75, 185]


array([(754.72107953, -0.26936706, -0.35238845, 73.17201984),
       (104.83282762, -1.11482256,  1.14303952, 20.66261802)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [29]:
# check myAnal_v3.preselection_v1 result
event_location = [21, 22]

for a, b in enumerate(event_location):
    print("event", a, b)
    sum_pt = np.sum(_presel_events_bef[a]['pT'] - presel_bef[b]['pT'])
    sum_eta = np.sum(_presel_events_bef[a]['eta'] - presel_bef[b]['eta'])
    sum_phi = np.sum(_presel_events_bef[a]['phi'] - presel_bef[b]['phi'])
    sum_m = np.sum(_presel_events_bef[a]['mass'] - presel_bef[b]['mass'])
    print(sum_pt, sum_eta, sum_phi, sum_m)
    print('-'*80)

event 0 21
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------
event 1 22
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------


##### Conclusion: The result of `preselection_v1` is the same as test version of event 21 and 22.

#### C. After $p_T > 20$ GeV and $\eta < 2.5$ selections

In [30]:
# test after pT and eta selections
_presel_events_pt_eta = []
_num_events_after_pt_eta = []

for i in range(21,23):
    print("event {}".format(i))
    _event_i = []
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        if (jet.pt > 20) and (abs(jet.eta) < 2.5):
            print(j, jet)
            _event_i.append((jet.pt, jet.eta, jet.phi, jet.mass))
    print('*'*20 + 'NumPy Array')
    # print('******', _event_i)  # [(pt1, eta1, phi1, m1), (pt2, eta2, ...), ...]
    arr_event_i = np.array(_event_i, dtype=[('pT', '<f8'), ('eta', '<f8'),
                                            ('phi', '<f8'), ('mass', '<f8')])
    print(arr_event_i.shape, arr_event_i)
    _num_events_after_pt_eta.append(len(arr_event_i))
    _presel_events_pt_eta.append(arr_event_i)
    print('-'*80)
    
print("Number of events after pT and eta preselection = {}".format(_num_events_after_pt_eta))
print('='*80)
_presel_events_pt_eta

event 21
0 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
1 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
2 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
4 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
5 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)
********************NumPy Array
(5,) [(754.72107953, -0.26936706, -0.35238845, 73.17201984)
 (104.83282762, -1.11482256,  1.14303952, 20.66261802)
 ( 85.25366785,  0.91831199, -2.53096755, 16.94153709)
 ( 54.1991487 ,  0.23044176, -2.59317889, 12.15247457)
 ( 27.54988231, -0.93685402,  0.65517858,  4.1833831 )]
--------------------------------------------------------------------------------
event 22
0 PseudoJet(pt=135.815, eta=2.146, phi=-3.118, mass=28.584)
1 PseudoJet(pt=23.674, eta=2.146, phi=-2.674, mass=8.452)
********************NumPy Array
(2,) [(135.81475624, 2.14646234, -3.11762065, 28.58375223)
 ( 23.67438139, 2.14635282, -2.67445185,  8.45196396)]
---------------------------

[array([(754.72107953, -0.26936706, -0.35238845, 73.17201984),
        (104.83282762, -1.11482256,  1.14303952, 20.66261802),
        ( 85.25366785,  0.91831199, -2.53096755, 16.94153709),
        ( 54.1991487 ,  0.23044176, -2.59317889, 12.15247457),
        ( 27.54988231, -0.93685402,  0.65517858,  4.1833831 )],
       dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')]),
 array([(135.81475624, 2.14646234, -3.11762065, 28.58375223),
        ( 23.67438139, 2.14635282, -2.67445185,  8.45196396)],
       dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])]

In [31]:
# check myAnal_v3.preselection_v1 result
event_location = [21, 22]

for a, b in enumerate(event_location):
    print("event", a, b)
    sum_pt = np.sum(_presel_events_pt_eta[a]['pT'] - presel_pt_eta[b]['pT'])
    sum_eta = np.sum(_presel_events_pt_eta[a]['eta'] - presel_pt_eta[b]['eta'])
    sum_phi = np.sum(_presel_events_pt_eta[a]['phi'] - presel_pt_eta[b]['phi'])
    sum_m = np.sum(_presel_events_pt_eta[a]['mass'] - presel_pt_eta[b]['mass'])
    print(sum_pt, sum_eta, sum_phi, sum_m)
    print('-'*80)

event 0 21
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------
event 1 22
0.0 0.0 0.0 0.0
--------------------------------------------------------------------------------


##### Conclusion: The result of `preselection_v1` is the same as test version of event 21 and 22.

### 6-1. Example 1
Print events

In [32]:
# print top 3 PseudoJet events in 1st, 2nd, and 3rd events
print(len(PseudoJet_filterDM_v3))
print('-'*80)
for i in range(3):
    print("event {}".format(i))
    print("How many PseudoJet does each event have = {}".format(len(PseudoJet_filterDM_v3[i])))
    print(PseudoJet_filterDM_v3[i][:3])
    print('-'*80)

19373
--------------------------------------------------------------------------------
event 0
How many PseudoJet does each event have = 88
[PseudoJet(pt=144.818, eta=0.390, phi=-0.246, mass=29.794), PseudoJet(pt=32.644, eta=0.658, phi=-0.752, mass=8.309), PseudoJet(pt=12.051, eta=-0.031, phi=-0.040, mass=1.622)]
--------------------------------------------------------------------------------
event 1
How many PseudoJet does each event have = 154
[PseudoJet(pt=156.632, eta=-0.721, phi=1.453, mass=24.608), PseudoJet(pt=116.277, eta=-1.841, phi=-2.920, mass=21.577), PseudoJet(pt=100.767, eta=-2.305, phi=-2.087, mass=12.047)]
--------------------------------------------------------------------------------
event 2
How many PseudoJet does each event have = 163
[PseudoJet(pt=180.419, eta=-1.263, phi=2.764, mass=34.022), PseudoJet(pt=13.794, eta=0.471, phi=1.933, mass=2.814), PseudoJet(pt=6.642, eta=-4.642, phi=1.803, mass=1.408)]
---------------------------------------------------------------

In [33]:
# use 21th event as example and print top 10 PseudoJet
PseudoJet_filterDM_v3[21][:10]

[PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172),
 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663),
 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942),
 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325),
 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152),
 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183),
 PseudoJet(pt=19.967, eta=-0.716, phi=-2.708, mass=3.603),
 PseudoJet(pt=11.095, eta=-1.614, phi=-3.022, mass=2.247),
 PseudoJet(pt=9.129, eta=-0.150, phi=2.344, mass=1.500),
 PseudoJet(pt=6.189, eta=-0.029, phi=-0.764, mass=2.306)]

In [34]:
# print pt > 20 GeV PseudoJet in 21th and 23th events
for i in range(21,23):
    print("event {}".format(i))
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        if jet.pt > 20:
            print(j, jet)
            
            # |eta| < 1.0 or 2.5
            # N_jet
            
            # pt1 > 440
            # pt2 > 150
            
            # [arr(event0), arr(event1), ...]
            # event0 = [[pt], [eta], [phi], [mass]] worse
            # event0 = [(pt1, eta1, phi1, mass1), (pt2, eta2, ), ...] better
            
    print('-'*80)

event 21
0 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
1 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
2 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
3 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325)
4 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
5 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)
--------------------------------------------------------------------------------
event 22
0 PseudoJet(pt=135.815, eta=2.146, phi=-3.118, mass=28.584)
1 PseudoJet(pt=23.674, eta=2.146, phi=-2.674, mass=8.452)
2 PseudoJet(pt=23.255, eta=3.290, phi=0.439, mass=4.790)
--------------------------------------------------------------------------------


In [35]:
# print jet elements in the detector level
for i in range(4):
    print(Jet.dataframelize(i))
    print('-'*80)

           Mass          PT       Eta       Phi
0  5.453704e+01  241.723770  0.370880 -0.282046
1 -9.536743e-07   26.896011 -1.431609  2.805379
--------------------------------------------------------------------------------
        Mass          PT       Eta       Phi
0   6.678744  177.236328 -0.139621 -0.447590
1  17.477238  118.888374 -0.711579  1.418930
2  19.143206  113.456596 -2.323572 -2.034082
3  20.718964  111.807510 -1.892479 -2.930634
--------------------------------------------------------------------------------
        Mass          PT       Eta       Phi
0  37.228954  188.137466 -1.268446  2.796516
1   3.147427   68.941101 -2.162593 -1.792415
2   1.937583   36.007828  1.437399  0.033540
3   5.678205   25.617086  2.068637 -0.297460
--------------------------------------------------------------------------------
        Mass          PT       Eta       Phi
0  20.538086  132.913483  1.756381 -0.577489
-------------------------------------------------------------------------

### 6-2. Example 2

In [36]:
for i in range(200):
    test_1, test_2 = [], []
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        if (jet.pt > 20):
            test_1.append((jet.pt, jet.eta, jet.phi, jet.mass))
            if (abs(jet.eta) < 2.5):
                test_2.append((jet.pt, jet.eta, jet.phi, jet.mass))
    if len(test_1) == 0:
        print('test 1', i, test_1, test_2)
    if len(test_2) == 0:
        print('test 2', i, test_1, test_2)
        print('-'*80)

test 1 28 [] []
test 2 28 [] []
--------------------------------------------------------------------------------
test 1 86 [] []
test 2 86 [] []
--------------------------------------------------------------------------------
test 2 100 [(84.17371578879752, 2.7916237519672458, -2.8580730931322766, 22.40237541012953)] []
--------------------------------------------------------------------------------
test 1 187 [] []
test 2 187 [] []
--------------------------------------------------------------------------------
test 1 188 [] []
test 2 188 [] []
--------------------------------------------------------------------------------


In [37]:
PseudoJet_filterDM_v3[100][:10]

[PseudoJet(pt=84.174, eta=2.792, phi=-2.858, mass=22.402),
 PseudoJet(pt=16.954, eta=1.607, phi=-1.019, mass=5.476),
 PseudoJet(pt=16.666, eta=3.050, phi=-2.073, mass=3.657),
 PseudoJet(pt=13.489, eta=-1.582, phi=2.140, mass=6.490),
 PseudoJet(pt=12.893, eta=-0.206, phi=-0.868, mass=5.059),
 PseudoJet(pt=9.855, eta=1.879, phi=-2.630, mass=4.359),
 PseudoJet(pt=9.811, eta=0.175, phi=0.013, mass=3.511),
 PseudoJet(pt=9.009, eta=0.093, phi=1.842, mass=3.726),
 PseudoJet(pt=8.390, eta=-1.855, phi=0.547, mass=3.861),
 PseudoJet(pt=8.365, eta=0.381, phi=-0.549, mass=2.833)]

## 7. Test 2: Constituents

In [38]:
# print attributes of each object
print(dir(PseudoJet_filterDM_v3))
print('-'*80)
print(dir(PseudoJet_filterDM_v3[21]))
print('-'*80)
print(dir(PseudoJet_filterDM_v3[21][0]))

['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
--------------------------------------------------------------------------------
['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__

In [39]:
# top 6 PseudoJet in 21th event
for j, jet in enumerate(PseudoJet_filterDM_v3[21][:6]):
    print(j, len(PseudoJet_filterDM_v3[21][j]), jet)

0 87 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
1 26 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
2 33 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
3 21 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325)
4 23 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
5 7 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)


In [40]:
# print the constituents of 6th PseudoJet in 21th event
print("event 5")
print(PseudoJet_filterDM_v3[21][5])
print('-'*80)
PseudoJet_filterDM_v3[21][5].constituents_array()

event 5
PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)
--------------------------------------------------------------------------------


array([(0.99460959, -1.1102134 , 0.67404842,  1.39569998e-01),
       (6.0019908 , -1.01715386, 0.54768401,  9.39570010e-01),
       (0.98741752, -0.776838  , 0.57899618,  1.39569998e-01),
       (1.61963058, -0.94204098, 0.55713117,  0.00000000e+00),
       (4.14160728, -0.93889606, 0.60915875, -8.42936970e-08),
       (7.85252857, -0.89518613, 0.74074847,  1.39569998e-01),
       (6.04252958, -0.88757044, 0.71782219,  9.38269973e-01)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [41]:
_presel_events_pt_eta[0]

array([(754.72107953, -0.26936706, -0.35238845, 73.17201984),
       (104.83282762, -1.11482256,  1.14303952, 20.66261802),
       ( 85.25366785,  0.91831199, -2.53096755, 16.94153709),
       ( 54.1991487 ,  0.23044176, -2.59317889, 12.15247457),
       ( 27.54988231, -0.93685402,  0.65517858,  4.1833831 )],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

##### Conclusion: My format of preselection events is the same as constituents of PseudoJet.

In [42]:
x = PseudoJet_filterDM_v3[21][5].constituents_array()
x['pT']

array([0.99460959, 6.0019908 , 0.98741752, 1.61963058, 4.14160728,
       7.85252857, 6.04252958])

In [43]:
a = PseudoJet_filterDM_v3[21][4].constituents_array()
b = PseudoJet_filterDM_v3[21][5].constituents_array()
print(a.shape)
print(b.shape)

(23,)
(7,)


In [44]:
# test to combine a(PseudoJet[21][4]) and b(PseudoJet[21][5])
c = np.array([a, b])
c[1]

array([(0.99460959, -1.1102134 , 0.67404842,  1.39569998e-01),
       (6.0019908 , -1.01715386, 0.54768401,  9.39570010e-01),
       (0.98741752, -0.776838  , 0.57899618,  1.39569998e-01),
       (1.61963058, -0.94204098, 0.55713117,  0.00000000e+00),
       (4.14160728, -0.93889606, 0.60915875, -8.42936970e-08),
       (7.85252857, -0.89518613, 0.74074847,  1.39569998e-01),
       (6.04252958, -0.88757044, 0.71782219,  9.38269973e-01)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [45]:
c

array([array([(1.80030811, -0.10393831, -2.54997563,  4.97610003e-01),
       (1.07274115,  0.12088498, -2.87783098,  4.93680000e-01),
       (4.73707819, -0.03949459, -2.6432538 ,  5.96046448e-08),
       (0.35985604,  0.04880412, -2.38159251,  5.10999991e-04),
       (3.47266984,  0.26385638, -2.32366896,  1.39569998e-01),
       (6.19313669,  0.06643461, -2.55104756,  8.42936970e-08),
       (1.61419094,  0.10874201, -2.5453608 , -2.10734243e-08),
       (0.54318136,  0.30021772, -2.50667334, -7.45058060e-09),
       (0.58041984,  0.451489  , -2.46868205,  0.00000000e+00),
       (4.03059387,  0.36115521, -2.43878222,  0.00000000e+00),
       (2.67526174,  0.40265584, -2.42060113, -2.98023224e-08),
       (0.87787473,  0.09467188, -2.67438483,  1.39569998e-01),
       (4.25370598,  0.20829731, -2.52334118,  1.39569998e-01),
       (1.76889956,  0.21774833, -2.5474999 ,  1.39569998e-01),
       (1.33180881,  0.43876269, -2.8383286 ,  1.49011612e-08),
       (0.83450967,  0.42169175, 

In [46]:
c[1][0]

(0.99460959, -1.1102134, 0.67404842, 0.13957)

In [47]:
b[0]

(0.99460959, -1.1102134, 0.67404842, 0.13957)

In [48]:
c[1]['pT'] - b['pT']

array([0., 0., 0., 0., 0., 0., 0.])

In [49]:
c['pT']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices